<a href="https://colab.research.google.com/github/DrorNeumeier/MyPalindromeMaker/blob/main/Drors_Palindromes_Maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Come join me in creating palindromes

#Step1: setup
Run the cell below by clicking the Play button on the code block.
This step gets the code from github and installs any required libraries

In [2]:

if not os.path.exists("MyPalindromeMaker"):
  !git clone https://github.com/DrorNeumeier/MyPalindromeMaker
else:
  !git -C MyPalindromeMaker pull 

!pip install transformers

import os
import random 
from transformers import BertTokenizer
from transformers import BertModel, BertConfig
import torch
import tensorflow as tf

print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

print("setting up the GPU devices that we will use in training later")
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

print("\n\n\nContinue to the next step...")

Cloning into 'MyPalindromeMaker'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 40 (delta 15), reused 14 (delta 3), pack-reused 0
Unpacking objects: 100% (40/40), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.2 MB/s 
     |████████████████████████████████| 7.6 MB 67.8 MB/s 
     |████████████████████████████████| 163 kB 59.8 MB/s 
Loading BERT tokenizer...


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

setting up the GPU devices that we will use in training later


SystemError: ignored

#Step2: The palindrome maker
Run the cell below by clicking the Play button on the code block.
This step builds my code to make palindromes.


In [61]:
import random

def loadWords(filename):
  f=open(filename)
  words=f.readlines()
  words=[word.strip().lower() for word in words]
  backwardWords = [word[::-1] for word in words]

  return words, backwardWords

def loadSentences(filename):
  f=open(filename)
  sentences=f.readlines()
  sentences=[sentence.strip().lower() for sentence in sentences]

  return sentences


def makeDictionary(wordList,currentString):

  #print("makeDictionary", currentString)

  ret={}
  ret["letters"] = {}

  letters = []
  newWordList=[]


  for word in wordList:
    if(word.startswith(currentString) and len(word)>len(currentString)):
      
      nextLetter=word[len(currentString)]
      newWordList.append(word)
      
      if nextLetter not in letters:
        letters.append(nextLetter)

  for letter in letters:
    ret["letters"][letter]=makeDictionary(newWordList, currentString+letter)

  if(currentString in wordList):
    ret["letters"][" "]={}

  return ret

def getDictionaryNode(dictionary, currentString):
  if len(currentString) == 0:
    return dictionary

  letter = currentString[0]
  currentString = currentString[1:]

  if letter not in dictionary["letters"].keys():
    return None

  return getDictionaryNode(dictionary["letters"][letter], currentString)


def makePalindrome(forwardDictionary, backwardDictionary, forwardString, backwardString, forwardList, backwardList, numberOfWords, printDebug=False):
  if printDebug:
    print(forwardList, forwardString,"...", backwardString, backwardList)
  
  forwardNode=getDictionaryNode(forwardDictionary, forwardString)
  backwardNode=getDictionaryNode(backwardDictionary, backwardString[::-1])

  if(len(forwardList)+len(backwardList) == numberOfWords-1 and (getDictionaryNode(forwardDictionary, forwardString+backwardString) != None and " " in getDictionaryNode(forwardDictionary, forwardString+backwardString)["letters"].keys())):
    forwardList.append(forwardString+backwardString)
    forwardString=""
    backwardString=""
    return forwardList+backwardList


  if(random.choice(list(forwardNode["letters"].keys()))==" "):
    forwardList.append(forwardString)
    forwardString=""
    forwardNode=forwardDictionary

  if(random.choice(list(backwardNode["letters"].keys()))==" "):
    backwardList.insert(0,backwardString)
    backwardString=""
    backwardNode=backwardDictionary

  if(len(forwardList)+len(backwardList) == numberOfWords and len(backwardString)==0 and len(forwardString)==0):
    return forwardList+backwardList

  if (len(forwardList)+len(backwardList) > numberOfWords):
      return None
  
  possibleLetters=(set(forwardNode["letters"].keys()) & set(backwardNode["letters"].keys())) - set(" ")
  possibleLetters = list(possibleLetters)

  #print(forwardNode["letters"].keys(), backwardNode["letters"].keys())
  #print(possibleLetters)

  if(len(possibleLetters)==0):
    return None

  
  random.shuffle(possibleLetters)
  for letter in possibleLetters:
    forwardString_=forwardString+letter
    backwardString_=letter+backwardString
    forwardList_= forwardList.copy()
    backwardList_=backwardList.copy()

    ret = makePalindrome(forwardDictionary, backwardDictionary, forwardString_, backwardString_, forwardList_, backwardList_, numberOfWords, printDebug=printDebug)
    if(ret):
      return ret


def tokenizeSentences(sentences):
  inputIds = []
  attentionMasks = []

  # For every sentence...
  for sentence in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer.encode_plus(
                          sentence,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          truncation=True,
                          max_length = 64,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          #padding=True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
        
      #The tokenized IDs
      inputIds.append(encoded_dict['input_ids'])
      #A mask with zeros past the end of the sentence 
      attentionMasks.append(encoded_dict['attention_mask'])

  return inputIds, attentionMasks

    
print("Continue to the next step...")


Continue to the next step...


#Step 3: Choose your dictionary and language reference
Select a dictionary from the drop downs to the right and run the cell below by clicking the Play button on the code block.<br>
This will load the selected dictionary.

In [44]:
dictionaryName = 'King James:kjv' #@param ["English:eng", "King James:kjv", "Sci Fi:sci"]
dictionaryPrefix = dictionaryName.split(":")[1]

print("loading dictionary", dictionaryName, "this can take a minute ro so")

words, backwardWords = loadWords("MyPalindromeMaker/dictionaries/%s_words.txt" % dictionaryPrefix)

sentences = loadSentences("MyPalindromeMaker/dictionaries/%s_sentences.txt" % dictionaryPrefix)

print("this dictionary has this many words in it:",len(words))

sentenceLenghts = {}
for sentence in sentences:
  wordCount = sentence.count(" ")

  if wordCount not in sentenceLenghts.keys():
    sentenceLenghts[wordCount] = 0
  
  sentenceLenghts[wordCount] = sentenceLenghts[wordCount] + 1

sentenceLenghtsKeys = list(sentenceLenghts)
sentenceLenghtsKeys.sort()

'''
print("This is the histogram to sentence lenght in this dictionary")
for sentenceLength in sentenceLenghtsKeys:
  print(sentenceLength, ":", sentenceLenghts[sentenceLength])
'''

forwardDictionary=makeDictionary(words, "")
backwardDictionary=makeDictionary(backwardWords, "")

print("dictionary is loaded. Lets try a test palindrome")

palindrome = makePalindrome(forwardDictionary, backwardDictionary, "", "", [], [], 5, printDebug=True)

print(" ".join(palindrome))





loading dictionary King James:kjv this can take a minute ro so
this dictionary has this many words in it: 12676
dictionary is loaded. Lets try a test palindrome
[]  ...  []
[] a ... a []
[] aj ... ja []
[] as ... sa []
[] asr ... rsa []
[] ass ... ssa []
[] assa ... assa []
[] ash ... hsa []
[] ashc ... chsa []
[] asa ... asa []
['asa'] m ... masa []
['asa'] ma ... amasa []
['asa'] mam ... m ['amasa']
['asa'] mamr ... rm ['amasa']
['asa'] maw ... w ['amasa']
['asa', 'maw'] a ... aw ['amasa']
['asa', 'maw'] an ... naw ['amasa']
['asa', 'maw'] ang ... gnaw ['amasa']
['asa', 'maw'] angl ... l ['gnaw', 'amasa']
['asa', 'maw'] angle ... el ['gnaw', 'amasa']
['asa', 'maw', 'angle'] h ... hel ['gnaw', 'amasa']
['asa', 'maw', 'angle'] ha ... ahel ['gnaw', 'amasa']
['asa', 'maw', 'angle'] has ... sahel ['gnaw', 'amasa']
['asa', 'maw', 'angle'] hasa ... asahel ['gnaw', 'amasa']
['asa', 'maw', 'angle'] had ... dahel ['gnaw', 'amasa']
['asa', 'maw', 'angle'] har ... rahel ['gnaw', 'amasa']
['asa',

#Now it is time to train the Natural Lang Processing model
You can pick the size of the training set to use on the right. We recomand to use 5000.<br>
Run the cell below by clicking the Play button on the code block.<br>
This will load the model and start the training. This can take a few minutes.

In [68]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig

trainingSetSize = 5000 #@param [5000, 10000, 20000]

trainingSetSize = int(trainingSetSize)
trainingSetSizeHalf = int(trainingSetSize/2)


print("First we need to create the training set")

trainingListSentences = random.choices(sentences, k=trainingSetSizeHalf)
trainingListLabels = [1] * (trainingSetSizeHalf)


#print(sentenceLenghts)
generatedSenteceLengths = random.choices(list(sentenceLenghts.keys()), weights=sentenceLenghts.values(), k=trainingSetSizeHalf)

for l in generatedSenteceLengths:
  if l <= 0 or l > 63:
    continue
  trainingListSentences.append(" ".join(random.choices(list(words), k=l)))
  trainingListLabels.append(0)

print("Here is a real sentence")
print("\"",trainingListSentences[0],"\"")
print("\nAnd here is a gibirish sentence")
print("\"",trainingListSentences[trainingSetSizeHalf], "\"")


print("\nand here is the tokenized sentence")
print('Tokenized: ', tokenizer.tokenize(trainingListSentences[0]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(trainingListSentences[0])))

inputIds, attentionMasks = tokenizeSentences(trainingListSentences)
# Convert the lists into tensors.
inputIds = torch.cat(inputIds, dim=0)
attentionMasks = torch.cat(attentionMasks, dim=0)
trainingListLabels = torch.tensor(trainingListLabels)

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(inputIds, attentionMasks, trainingListLabels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

#this is the actual model we will be training
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()


First we need to create the training set
Here is a real sentence
" and joshua the son of nun was full of the spirit of wisdom "

And here is a gibirish sentence
" mastery grate sucking pibeseth gaal sarais stayeth "

and here is the tokenized sentence
Tokenized:  ['and', 'joshua', 'the', 'son', 'of', 'nun', 'was', 'full', 'of', 'the', 'spirit', 'of', 'wisdom']
Token IDs:  [1998, 9122, 1996, 2365, 1997, 16634, 2001, 2440, 1997, 1996, 4382, 1997, 9866]
5000
4,500 training samples
  500 validation samples


In [ ]:
for word in words:
  if word.startswith("unnum"):
    print(word)

unnumbered
